In [19]:
#Access to my Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
import pandas as pd

In [21]:
import sqlite3

In [22]:
df = pd.read_csv('drive/MyDrive/SQLpractice/P9-OLTP.csv')
df.head()


,TransactionID,Timestamp,CustomerID,Name,Surname,Shipping_State,Item,Description,Retail_Price,Loyalty_Discount
0,113255,2016-12-13 18:05:37,200000663,Leonard,Jackson,Delaware,8619953091,Pillowcase,18.96,0.02
1,113254,2016-12-13 18:05:22,200000661,Lucas,Abraham,Louisiana,2020397001,Men's Pajama Set,23.10,0.00
2,113253,2016-12-13 18:03:57,200000662,Trevor,Skinner,Louisiana,4681342313,Sheets,29.90,0.10
3,113252,2016-12-13 18:01:45,100000007,Dorothy,Thomson,Alabama,6697166886,Coat,159.80,0.07
4,113251,2016-12-13 18:01:22,400000180,Eric,Bower,New Hampshire,6697166886,Coat,159.80,0.06


In [23]:
df.shape

(3455, 10)

In [24]:
unique_values = df.nunique()
unique_values

TransactionID       3455
Timestamp           3434
CustomerID           942
Name                 165
Surname              150
Shipping_State        50
Item                 126
Description           68
Retail_Price         110
Loyalty_Discount      11
dtype: int64

**Note**: Each transaction is unique, it can be concluded that only one item is sold in each transaction

In [25]:
df.describe()

,TransactionID,CustomerID,Item,Retail_Price,Loyalty_Discount
count,3455.000000,3.455000e+03,3.455000e+03,3455.000000,3455.000000
mean,111528.000000,1.797979e+08,5.276712e+09,58.526237,0.050457
std,997.516917,9.563412e+07,2.600486e+09,34.464217,0.032215
min,109801.000000,1.000000e+08,1.039855e+09,5.600000,0.000000
25%,110664.500000,1.000003e+08,2.963301e+09,31.800000,0.020000
50%,111528.000000,1.000009e+08,5.145202e+09,51.660000,0.050000
75%,112391.500000,2.000009e+08,7.645689e+09,79.800000,0.080000
max,113255.000000,4.000009e+08,9.916068e+09,159.800000,0.100000


In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3455 entries, 0 to 3454
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   TransactionID     3455 non-null   int64  
 1   Timestamp         3455 non-null   object 
 2   CustomerID        3455 non-null   int64  
 3   Name              3455 non-null   object 
 4   Surname           3455 non-null   object 
 5   Shipping_State    3455 non-null   object 
 6   Item              3455 non-null   int64  
 7   Description       3455 non-null   object 
 8   Retail_Price      3455 non-null   float64
 9   Loyalty_Discount  3455 non-null   float64
dtypes: float64(2), int64(3), object(5)
memory usage: 270.0+ KB


In [27]:
#Null values
df.isna().sum()

TransactionID       0
Timestamp           0
CustomerID          0
Name                0
Surname             0
Shipping_State      0
Item                0
Description         0
Retail_Price        0
Loyalty_Discount    0
dtype: int64

**Note** : The dataset contains information on products, clients, and transactions, requiring the creation of three tables to accommodate this information.

## Creating my SQL database

In [28]:
#Set path to database file in Google Drive
db_path = '/content/drive/MyDrive/SQLpractice/store_sales.db'

In [29]:
#Create a database for our store sales dataset
conn = sqlite3.connect(db_path)

In [30]:
#Create table customer
cursor = conn.cursor()
cursor.execute('''CREATE TABLE customer (
  CustomerID INT NOT NULL PRIMARY KEY,
  Name CHAR(30) NOT NULL,
  Surname CHAR(30) NOT NULL,
  Loyalty_Discount FLOAT(1,6) DEFAULT '0.000000'
  ); '''
  )
conn.commit()
cursor.close()

In [31]:
#Create table item
cursor = conn.cursor()
cursor.execute('''CREATE TABLE item (
  ItemID INT NOT NULL PRIMARY KEY,
  Description VARCHAR(255) NOT NULL,
  Retail_Price FLOAT(9,6) NOT NULL
  ); '''
  )
conn.commit()
cursor.close()

In [32]:
#Create table transaction
cursor = conn.cursor()
conn.execute('''CREATE TABLE transactions (
  TransactionID INT NOT NULL PRIMARY KEY,
  Timestamp DATETIME DEFAULT CURRENT_TIMESTAMP,
  CustomerID INT NOT NULL,
  ItemID INT NOT NULL,
  Shipping_State VARCHAR(255) NOT NULL,
  CONSTRAINT fk_Customer_id FOREIGN KEY (CustomerID) REFERENCES costumer(CostumerID),
  CONSTRAINT fk_Item_id FOREIGN KEY (ItemID) REFERENCES item(ItemID)
  ); '''
  )
conn.commit()
cursor.close()

In [33]:
# Create a cursor object
cursor = conn.cursor()

# Execute the SQL query to get table names
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

# Fetch all the table names
tables = cursor.fetchall()

# Print the table names
for table in tables:
    print(table[0])

# Close the cursor and the database connection
cursor.close()

customer
item
transactions


## Formatting data for insertion into the database

In [34]:
#Changing the name of column Item
df = df.rename(columns={'Item': 'ItemID'})
df.columns

Index(['TransactionID', 'Timestamp', 'CustomerID', 'Name', 'Surname',
       'Shipping_State', 'ItemID', 'Description', 'Retail_Price',
       'Loyalty_Discount'],
      dtype='object')

In [35]:
#get data to populate table costumer 
customer_data = df[['CustomerID', 'Name', 'Surname', 'Loyalty_Discount']]

In [36]:
customer_data.shape

(3455, 4)

In [37]:
# Making each customer unique
customer_data = customer_data.drop_duplicates(subset=['CustomerID'])

In [38]:
customer_data.shape

(942, 4)

In [39]:
#get data to populate table item
item_data = df[['ItemID', 'Description', 'Retail_Price']]

In [40]:
item_data.shape

(3455, 3)

In [41]:
#Making each item unique
item_data = item_data.drop_duplicates(subset=['ItemID'])

In [42]:
item_data.shape

(126, 3)

In [43]:
#get data to populate table transactions
transactions_data = df[['TransactionID', 'Timestamp', 'CustomerID', 'ItemID', 'Shipping_State']]

**Note:** Every transaction is unique

## Populating the database

In [44]:
#Populate table client
customer_data.to_sql('customer', conn, if_exists='append', index=False)

# Commit changes
conn.commit()

In [45]:
cursor = conn.cursor()

cursor.execute('SELECT * FROM customer')

# Fetch all the results as a list of tuples
results = cursor.fetchall()

# Print the results
for row in results:
    print(row)

(200000663, 'Leonard', 'Jackson', 0.02)
(200000661, 'Lucas', 'Abraham', 0.0)
(200000662, 'Trevor', 'Skinner', 0.1)
(100000007, 'Dorothy', 'Thomson', 0.07)
(400000180, 'Eric', 'Bower', 0.06)
(200000701, 'Joshua', 'Wilson', 0.1)
(300000467, 'Angela', 'Morrison', 0.08)
(400000706, 'Samantha', 'Mills', 0.1)
(100000061, 'Kylie', 'Howard', 0.1)
(400000513, 'Amy', 'Wilkins', 0.02)
(200000459, 'Charles', 'Brown', 0.04)
(200000482, 'Justin', 'Graham', 0.01)
(100000081, 'Joan', 'Buckland', 0.02)
(200000821, 'Charles', 'Underwood', 0.08)
(300000015, 'Madeleine', 'Marshall', 0.08)
(100000252, 'Nicholas', 'Lyman', 0.09)
(300000822, 'Leonard', 'Fisher', 0.07)
(300000815, 'Pippa', 'Robertson', 0.03)
(100000618, 'Andrew', 'Wright', 0.08)
(100000051, 'Sophie', 'Wilkins', 0.04)
(100000880, 'Rachel', 'Springer', 0.03)
(100000898, 'Samantha', 'Ross', 0.02)
(400000359, 'Gabrielle', 'MacDonald', 0.07)
(100000559, 'Grace', 'Mills', 0.09)
(100000878, 'Felicity', 'Metcalfe', 0.03)
(100000013, 'Ryan', 'MacDonal

In [46]:
#Populate table item
item_data.to_sql('item', conn, if_exists='append', index=False)

# Commit changes
conn.commit()

In [47]:
cursor = conn.cursor()

cursor.execute('SELECT * FROM item')

# Fetch all the results as a list of tuples
results = cursor.fetchall()

# Print the results
for row in results:
    print(row)

(8619953091, 'Pillowcase', 18.96)
(2020397001, "Men's Pajama Set", 23.1)
(4681342313, 'Sheets', 29.9)
(6697166886, 'Coat', 159.8)
(5187373849, 'Short Slip', 12.87)
(7010679527, 'Blanket', 63.84)
(1893735141, 'Socks', 115.5)
(8241463559, 'Wallet', 49.75)
(2430071492, 'Dust Ruffle', 41.86)
(6988128169, 'Handbag', 25.44)
(7270957383, 'Scarf', 29.85)
(6228611532, 'Valance', 47.76)
(1940915493, 'Long Gown', 68.77)
(3305177499, 'Coveralls (long sleeve)', 95.88)
(4924918902, 'T-shirt', 62.79)
(2363094138, 'Short Overalls', 71.82)
(2355020550, 'Sweater', 67.92)
(3890778939, 'Suit & 2 pc Dress', 125.16)
(5153370805, 'Shorts', 43.96)
(1737807900, 'Tie', 68.64)
(2695940506, 'Sweater', 59.4)
(9873264896, 'Hat', 10.89)
(8068199275, 'Slacks', 87.78)
(2588421965, 'Commode Cover', 15.84)
(9730446421, 'Blazer', 103.3)
(4327321762, 'Dress', 69.75)
(3126356136, 'Formal Dress', 99.9)
(2178891798, 'Hand Towel', 18.17)
(3795753080, 'Jeans', 38.1)
(2210750188, 'Blazer', 119.76)
(5509524170, 'Couch Cover', 86

In [48]:
#Populate table transactions
transactions_data.to_sql('transactions', conn, if_exists='append', index=False)

# Commit changes
conn.commit()

In [49]:
cursor = conn.cursor()

cursor.execute('SELECT * FROM transactions')

# Fetch all the results as a list of tuples
results = cursor.fetchall()

# Print the results
for row in results:
    print(row)

(113255, '2016-12-13 18:05:37', 200000663, 8619953091, 'Delaware')
(113254, '2016-12-13 18:05:22', 200000661, 2020397001, 'Louisiana')
(113253, '2016-12-13 18:03:57', 200000662, 4681342313, 'Louisiana')
(113252, '2016-12-13 18:01:45', 100000007, 6697166886, 'Alabama')
(113251, '2016-12-13 18:01:22', 400000180, 6697166886, 'New Hampshire')
(113250, '2016-12-13 17:59:20', 200000701, 5187373849, 'North Carolina')
(113249, '2016-12-13 17:59:11', 300000467, 7010679527, 'Utah')
(113248, '2016-12-13 17:58:07', 400000706, 1893735141, 'Alabama')
(113247, '2016-12-13 17:57:50', 100000061, 8241463559, 'Minnesota')
(113246, '2016-12-13 17:55:30', 400000513, 2430071492, 'Minnesota')
(113245, '2016-12-13 17:51:34', 200000459, 6988128169, 'Arizona')
(113244, '2016-12-13 17:48:55', 200000482, 7270957383, 'Arizona')
(113243, '2016-12-13 17:48:05', 100000081, 6228611532, 'Minnesota')
(113242, '2016-12-13 17:46:18', 200000821, 1940915493, 'Texas')
(113241, '2016-12-13 17:45:21', 300000015, 3305177499, 'A